<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Frameworks%20and%20agents/langchain_sql_agents_withfunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchainhub
!pip install google-search-results
!pip install langchain openai sqlalchemy
!pip install -U langchain-community

In [ ]:
from langchain.agents.load_tools import load_tools
from langchain.tools import tool
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI
from typing import Union
import pandas as pd


In [ ]:
from google.colab import drive, userdata,  output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
@tool
def convert_currency(USD_amount: float, currency: str) -> Union[float, str]:
    """ Converts an amount in US dollars to another currency.

    Args:
        USD_amount: amount in US dollars.
        currency: name of target currency (e.g., "Yen").

    Returns:
        input amount in target currency.
    """
    conversion_factors = {
            'Euro': 0.92,       # 1 USD = 0.92 EUR
            'Yen': 156.23,      # 1 USD = 156.23 JPY
            'Yuan': 7.24,       # 1 USD = 7.24 CNY
            'Pound': 0.79,      # 1 USD = 0.79 GBP
            'Won': 1364.55,     # 1 USD = 1364.55 KRW
            'Rupee': 83.28      # 1 USD = 83.28 INR
        }

    if currency not in conversion_factors:
        error_message = (
            f'Unknown currency: {currency}!'
            f'Use one of {conversion_factors.keys()}')
        return error_message

    conversion_factor = conversion_factors[currency]
    converted_amount = USD_amount * conversion_factor
    return converted_amount

In [ ]:
llm = ChatOpenAI(api_key = userdata.get('OPENAI_API_KEY'),temperature=0, model='gpt-4o')
db_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/structured_data/games.db'


db = SQLDatabase.from_uri(f'sqlite:///{db_path}')
extra_tools = load_tools(
    ['serpapi'], serpapi_api_key="4efe4e98e408d433c0d2a842e320df23743b16d813d5fce429b5665367c7193c", llm=llm)
extra_tools.append(convert_currency)

agent = create_sql_agent(
    llm=llm, db=db, verbose=True,
    agent_type='openai-tools',
    extra_tools=extra_tools)
agent.invoke({'input':
 'What revenue was generated by computer games in 2015 ? How much is it in euros?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


games
Invoking: `sql_db_schema` with `{'table_names': 'games'}`



CREATE TABLE games (
	rank INTEGER, 
	name TEXT, 
	platform TEXT, 
	year INTEGER, 
	genre TEXT, 
	publisher TEXT, 
	americasales NUMERIC, 
	eusales NUMERIC, 
	japansales NUMERIC, 
	othersales NUMERIC, 
	globalsales NUMERIC
)

/*
3 rows from games table:
rank	name	platform	year	genre	publisher	americasales	eusales	japansales	othersales	globalsales
1	Wii Sports	Wii	2006	Sports	Nintendo	41.4900000000	29.0200000000	3.7700000000	8.4600000000	82.7400000000
2	Super Mario Bros.	NES	1985	Platform	Nintendo	29.0800000000	3.5800000000	6.8100000000	0.7700000000	40.2400000000
3	Mario Kart Wii	Wii	2008	Racing	Nintendo	15.8500000000	12.8800000000	3.7900000000	3.3100000000	35.8200000000
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT SUM(globalsales) AS total_revenue_2015 FROM games WHERE year = 2015 AND platform = 'PC'"}`


```sql
SELEC

{'input': 'What revenue was generated by computer games in 2015 ? How much is it in euros?',
 'output': 'The revenue generated by computer games on the PC platform in 2015 was approximately $8.07 million, which is about €7.42 million.'}